In [ ]:
import pandas as pd
import plotly.express as px
import numpy as np
import os

segment = 'method' # 'contract' # 'project'

In [ ]:
prepend = 'csv_exports/whale_actions/'

op_df = pd.read_csv(prepend + 'op_whale_actions_22_01_18.csv')
op_df['chain'] = 'Optimism'

l1_df = pd.read_csv(prepend + 'l1_whale_actions_22_01_18.csv')
l1_df['chain'] = 'Ethereum'

arb_df = pd.read_csv(prepend + 'arbi_whale_actions_22_01_18.csv')
arb_df['chain'] = 'Arbitrum'

df = pd.concat([op_df, l1_df, arb_df])
df = df[df['project'] != '0x0000000000000000000000000000000000000000']
df = df.fillna(0)

# df.sample(20)



In [ ]:
df_merge = df.copy()
df_merge['usd_value_in'] = np.where(df_merge['token_direction'] == 'in',df_merge['usd_value'], 0).astype(np.float64)
df_merge['usd_value_out'] = np.where(df_merge['token_direction'] == 'out',df_merge['usd_value'], 0).astype(np.float64)
df_merge = df_merge.groupby(['chain','address','project','contract','method']).sum()
df_merge = df_merge.reset_index()

df_merge['usd_value_net'] =  df_merge['usd_value_out'] - df_merge['usd_value_in'] # net in contracts
df_merge['usd_value_net'] = np.where(df_merge['usd_value_net'] < 0, 0, df_merge['usd_value_net'])
val_cols = ['usd_value','usd_value_in','usd_value_out','usd_value_net']
display(df_merge.sample(20))

In [ ]:
df_addr = df_merge.groupby(['chain','address','project','contract','method']).sum()
df_addr.reset_index(inplace=True)

for val in val_cols:
        df_addr[val + '_pct_share'] = df_addr.groupby(['address','chain'])[val].transform(lambda x: x / x.sum())
display(df_addr.sample(20))

In [ ]:
group_list = ['chain','project','contract','method']
dlist = ['chain','project','contract','method','action'\
                ,'usd_value','usd_value_in','usd_value_out','usd_value_net'\
                ,'usd_value_pct_share','usd_value_in_pct_share','usd_value_out_pct_share','usd_value_net_pct_share'
                ,'address' \
                ]

if segment == 'project':
        group_list.drop('method',inplace=True)
        group_list.drop('contract',inplace=True)
        dlist.drop('method',inplace=True)
        dlist.drop('contract',inplace=True)
elif segment == 'contract':
        group_list.drop('method',inplace=True)
        dlist.drop('method',inplace=True)

In [ ]:
df_dollar = df_merge[['chain','project','contract','method','usd_value','usd_value_in','usd_value_out','usd_value_net','address']]
df_dollar = df_dollar.groupby(['chain','project','contract','method']).agg({
        'usd_value':'sum'
        ,'usd_value_in':'sum'
        ,'usd_value_out':'sum'
        ,'usd_value_net':'sum'
        ,'address':'nunique'
})
df_dollar.reset_index(inplace=True)
df_dollar.name = 'df_dollar'
for val in val_cols:
        df_dollar[val + '_pct_share'] =  df_dollar[val] / ( df_dollar[val].sum() )


In [ ]:
# Get the average volume share across all whales
# i.e. once someone is determined to be a whale, tkae their pct transfer share, and then we avg across rather than $ weight.
# The reason to not dollar-weight is that we care more about genetating the average individual user experience, rather than the whaliest whale.

df_action = df_addr.groupby(['chain','project','contract','method']).agg({
        'usd_value_pct_share':'mean'
        ,'usd_value_in_pct_share':'mean'
        ,'usd_value_out_pct_share':'mean'
        ,'usd_value_net_pct_share':'mean'
        ,'usd_value':'mean'
        ,'usd_value_in':'mean'
        ,'usd_value_out':'mean'
        ,'usd_value_net':'mean'
        ,'address':'nunique'
})

df_action_by_app = df_addr.groupby(['chain','project','address']).agg({
        # 'usd_value_pct_share':'mean'
        # ,'usd_value_in_pct_share':'mean'
        # ,'usd_value_out_pct_share':'mean'
        # ,'usd_value_net_pct_share':'mean'
        'usd_value':'sum'
        ,'usd_value_in':'sum'
        ,'usd_value_out':'sum'
        ,'usd_value_net':'sum'
})

df_action_by_app.reset_index(inplace=True)

for val in val_cols:
        df_action_by_app[val + '_pct_share'] = df_action_by_app.groupby(['chain','address'])[val].transform(lambda x: x / x.sum())

df_action_by_app = df_action_by_app.groupby(['chain','project']).agg({
        'usd_value_pct_share':'mean'
        ,'usd_value_in_pct_share':'mean'
        ,'usd_value_out_pct_share':'mean'
        ,'usd_value_net_pct_share':'mean'
        ,'usd_value':'mean'
        ,'usd_value_in':'mean'
        ,'usd_value_out':'mean'
        ,'usd_value_net':'mean'
        ,'address':'nunique'
})


df_action.reset_index(inplace=True)
df_action_by_app.reset_index(inplace=True)

df_action.name = 'df_action'
df_action_by_app.name = 'df_action_by_app'

# display(df_action[df_action['project'] == 'gmx'])
# display(df_action_by_app[df_action_by_app['project'] == 'gmx'])

In [ ]:
dfs = [df_action, df_dollar]


if not os.path.exists("csv_outputs"):
        os.mkdir("csv_outputs")
        
for i, d in enumerate(dfs):
        original_name = d.name
        d.reset_index(inplace=True)
        if segment == 'method':
                d['action'] = d['project'].astype(str) + ' - ' + d['contract'].astype(str) + ' | ' + d['method'].astype(str)
        elif segment == 'contract':
                d['action'] = d['project'].astype(str) + ' - ' + d['contract'].astype(str)
        elif segment == 'project':
                d['action'] = d['project'].astype(str)
        
        # d = d[dlist]
        
        dfs[i] = d
        d.name = original_name
        
#dumb hardcode, tried to do it fancy, but whatever
df_action = dfs[0]
df_dollar = dfs[1]

df_action.to_csv('csv_outputs/whale_actions_share.csv')
df_dollar.to_csv('csv_outputs/whale_actions_share_dollar_weight.csv')

df_action.sample(20)
# gix = px.pie(df_action, values='usd_value_out_pct_share', names='action', title='Share of Actions')

In [ ]:
final_cols = ['chain','project','action','usd_value_out','usd_value_out_pct_share','usd_value_net','usd_value_net_pct_share','address','style','granularity']

In [ ]:
style = 'Dollar-Weighted'
print('Dollar-Weighted')
chains = df_dollar['chain'].drop_duplicates().to_list()
# print('Top Dollar Weighted Actions by Chain')
dfs = []
for c in chains:
        tdf = df_dollar[df_dollar['chain'] == c]
        sumdf = []
        print('Sum by Action')
        sumdf = tdf.groupby(['chain','project','action']).sum()
        sumdf.reset_index(inplace=True)
        
        for val in val_cols:
                sumdf[val + '_pct_share'] = sumdf.groupby(['chain'])[val].transform(lambda x: x / x.sum())
        
        # sumdf['usd_value_net_pct_share_100'] = sumdf['usd_value_net_pct_share'] * 100
        # sumdf['usd_value_out_pct_share_100'] = sumdf['usd_value_out_pct_share'] * 100
        sumdf = sumdf.sort_values(by = 'usd_value_out_pct_share', ascending = False)
        sumdf['style'] = style
        sumdf['granularity'] = 'action'
        sumdf = sumdf[final_cols]
        # display( sumdf.head(10) )
        # display(sumdf.head())
        dfs.append(sumdf)

        print('Sum by App')
        sumdf = []
        sumdf = tdf.groupby(['chain','project']).sum()
        sumdf.reset_index(inplace=True)
        for val in val_cols:
                sumdf[val + '_pct_share'] = sumdf.groupby(['chain'])[val].transform(lambda x: x / x.sum())

        # sumdf['usd_value_net_pct_share_100'] = sumdf['usd_value_net_pct_share'] * 100
        # sumdf['usd_value_out_pct_share_100'] = sumdf['usd_value_out_pct_share'] * 100
        sumdf = sumdf.sort_values(by = 'usd_value_out_pct_share', ascending = False)
        sumdf['style'] = style
        sumdf['action'] = sumdf['project']
        sumdf['granularity'] = 'project'
        sumdf = sumdf[final_cols]
        # display( sumdf.head(10) )
        # display(sumdf.head())
        dfs.append(sumdf)
        # print(sumdf[['usd_value_out_pct_share_100']].sum())

In [ ]:
style = 'Address-Weighted'
print(style)
# print(df_action.columns)
for c in chains:
        tdf = df_action[df_action['chain'] == c]
        # display(tdf)
        tdf = tdf[tdf['address'] >=20]
        sumdf = tdf
        print('Sum by Action')
        # sumdf = tdf.groupby(['chain','project','action']).sum()
        # for val in val_cols:
        #         sumdf[val + '_pct_share'] = sumdf.groupby(['chain'])[val].transform(lambda x: x / x.sum())
        sumdf = tdf.groupby(['chain','project','action']).mean()
        sumdf.reset_index(inplace=True)
        # print(sumdf.columns)
        # sumdf['usd_value_net_pct_share_100'] = sumdf['usd_value_net_pct_share'] * 100
        # sumdf['usd_value_out_pct_share_100'] = sumdf['usd_value_out_pct_share'] * 100
        sumdf = sumdf.sort_values(by = 'usd_value_out_pct_share', ascending = False)
        sumdf['style'] = style
        sumdf['granularity'] = 'action'
        sumdf = sumdf[final_cols]
        # display( sumdf.head(10) )
        # display(sumdf.head())
        dfs.append(sumdf)

        print('Sum by App')
        sumdf = df_action_by_app[df_action_by_app['chain'] == c]
        sumdf = sumdf[sumdf['address'] >=20]
        # for val in val_cols:
        #         sumdf[val + '_pct_share'] = sumdf.groupby(['chain'])[val].transform(lambda x: x / x.sum())
        sumdf = sumdf.groupby(['chain','project']).mean()
        sumdf.reset_index(inplace=True)
        # sumdf['usd_value_net_pct_share_100'] = sumdf['usd_value_net_pct_share'] * 100
        # sumdf['usd_value_out_pct_share_100'] = sumdf['usd_value_out_pct_share'] * 100
        sumdf = sumdf.sort_values(by = 'usd_value_out_pct_share', ascending = False)
        sumdf['style'] = style
        sumdf['action'] = sumdf['project']
        sumdf['granularity'] = 'project'
        sumdf = sumdf[final_cols]

        # display(sumdf.head())
        dfs.append(sumdf)
        # display( sumdf.head(10) )
        # print(sumdf[['usd_value_out_pct_share_100']].sum()) #SHould be > 100 since it's avg by address

In [ ]:
pd_export = pd.concat(dfs)
pd_export = pd_export[pd_export['usd_value_out_pct_share'] > (0.01 / 100) ] # greater than 0.01 %
pd_export.to_csv('csv_outputs/whale_agg_summary.csv')
display(pd_export)